# Tests for `predictor.utils`

## How to use:

Check your current kernel, if it's not the project's venv, change it:

* Click the top right button to change the kernel to your project's venv (e.g. `'mobilidade_rio_api' (Python 3.10.8)` ([learn more](https://code.visualstudio.com/docs/python/jupyter-support-py#_connect-to-a-remote-jupyter-server)).

# Code

Imports

In [1]:
import init_django
import pandas as pd
# django
# from mobilidade_rio.utils import query_utils as qu
# from mobilidade_rio.utils import jupyter_utils as ju
# from mobilidade_rio.utils import django_utils as du
from mobilidade_rio.pontos.models import *
from mobilidade_rio.predictor.models import *
# query
from django.db import connection
cur = connection.cursor() 

test task

In [2]:
from mobilidade_rio.config_django_q import tasks
tasks.generate_prediction()

[GENPRED] etapa 1
[GENPRED] etapa 2
[GENPRED] etapa 3
[GENPRED] 3.1
[GENPRED] 3.2
[GENPRED] 3.3
[GENPRED] 3.4
[GENPRED] 3.4 0
[GENPRED] 3.5
[GENPRED] 3.6
[GENPRED] 3.7 insert [{'trip_id': '2000014113020102', 'stop_id': '424132201001372', 'id_veiculo': 'B53', 'latitude': -23.00812, 'longitude': -43.44229, 'dataHora': Timestamp('2023-01-27 20:15:44.515169'), 'trip_short_name': '14', 'direction_id': 1, 'service_id': 'U', 'stop_sequence': 1, 'arrival_time': None}, {'trip_id': '2000014113010102', 'stop_id': '519148202000372', 'id_veiculo': nan, 'latitude': -22.918365, 'longitude': -43.639274, 'dataHora': Timestamp('2023-01-27 20:15:44.515169'), 'trip_short_name': '14', 'direction_id': 0, 'service_id': 'U', 'stop_sequence': 1, 'arrival_time': None}]


IntegrityError: null value in column "arrival_time" of relation "predictor_prediction" violates not-null constraint
DETAIL:  Failing row contains (2, 2000014113020102, 424132201001372, B53, -23.00812, -43.44229, 2023-01-27 20:15:44.515169, 14, 1, U, 1, null).


Add df to django ORM

In [3]:
import hashlib

Agency.objects.all().delete()
# create df based on model
new_agency = Agency(
    agency_name = 'a1',
)
df_agency = pd.DataFrame([new_agency.__dict__])
df_agency = df_agency.drop(['_state'], axis=1)
# set random pk
random_md5 = hashlib.md5(os.urandom(128)).hexdigest()
df_agency['agency_id'] = random_md5

# add 3 random items
for i in range(3):
    random_md5 = hashlib.md5(os.urandom(128)).hexdigest()
    # concat with new values
    df_agency = pd.concat([df_agency, pd.DataFrame([{'agency_id': random_md5}])], ignore_index=True)

display(df_agency)
for row in df_agency.to_dict('records'):
    print("row",row)
#     Agency.objects.create(**row)

# bulk create
Agency.objects.bulk_create([Agency(**row) for row in df_agency.to_dict('records')])

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_branding_url,agency_fare_url,agency_email
0,999427adac028028a36ac4210d42a959,a1,,,None,None,None,None,None
1,35f115ba54f89222da2cc0d1a3074d30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3e4840d776769235ed1349561695448e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,fbf97e9c5f8f75a3203efe06bb447065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


row {'agency_id': '999427adac028028a36ac4210d42a959', 'agency_name': 'a1', 'agency_url': '', 'agency_timezone': '', 'agency_lang': None, 'agency_phone': None, 'agency_branding_url': None, 'agency_fare_url': None, 'agency_email': None}
row {'agency_id': '35f115ba54f89222da2cc0d1a3074d30', 'agency_name': nan, 'agency_url': nan, 'agency_timezone': nan, 'agency_lang': nan, 'agency_phone': nan, 'agency_branding_url': nan, 'agency_fare_url': nan, 'agency_email': nan}
row {'agency_id': '3e4840d776769235ed1349561695448e', 'agency_name': nan, 'agency_url': nan, 'agency_timezone': nan, 'agency_lang': nan, 'agency_phone': nan, 'agency_branding_url': nan, 'agency_fare_url': nan, 'agency_email': nan}
row {'agency_id': 'fbf97e9c5f8f75a3203efe06bb447065', 'agency_name': nan, 'agency_url': nan, 'agency_timezone': nan, 'agency_lang': nan, 'agency_phone': nan, 'agency_branding_url': nan, 'agency_fare_url': nan, 'agency_email': nan}


[<Agency: Agency object (999427adac028028a36ac4210d42a959)>,
 <Agency: Agency object (35f115ba54f89222da2cc0d1a3074d30)>,
 <Agency: Agency object (3e4840d776769235ed1349561695448e)>,
 <Agency: Agency object (fbf97e9c5f8f75a3203efe06bb447065)>]

In [8]:
queryset = Agency.objects.all()
# load df
df = pd.DataFrame.from_records(queryset.values())
df

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_branding_url,agency_fare_url,agency_email
0,999427adac028028a36ac4210d42a959,a1,,,None,None,None,None,None
1,35f115ba54f89222da2cc0d1a3074d30,nan,nan,nan,nan,nan,nan,nan,nan
2,3e4840d776769235ed1349561695448e,nan,nan,nan,nan,nan,nan,nan,nan
3,fbf97e9c5f8f75a3203efe06bb447065,nan,nan,nan,nan,nan,nan,nan,nan


queryset to df